# Running BOA Optimization Directly in Python

This notebook demonstrates how to:

Write a basic Wrapper in Python and launch a optimization from Python.
If you wanted to launch it from command line, you would do a similar thing of defining the Wrapper, and then put in your configuration file the information about where the wrapper is, and use [BOA's](../index.rst) CLI tools. See [Running an Experiment from Command Line (Python Wrapper)](example_py_run.rst) for more information.

In [1]:
import pathlib

import yaml
import shutil

import boa
from wrapper import Wrapper

In [2]:
old_runs = pathlib.Path().resolve().glob("boa_runs*")
for path in old_runs:
    shutil.rmtree(path)

## Loading the Config File

In [3]:
config_path = pathlib.Path().resolve() / "single_config.yaml"

Here we can see what the configuration file looks like

In [12]:
with open(config_path, 'r') as f:
    file_contents = f.read()
    print (file_contents)

# Single objective optimization config
optimization_options:
    objective_options:
        objectives:
            - name: Cosine8
    trials: 20
    append_timestamp: False
parameters:
    x0:
        type: range
        bounds: [0.0, 1.0]
    x1:
        type: range
        bounds: [0.0, 1.0]
    x2:
        type: range
        bounds: [0.0, 1.0]
    x3:
        type: range
        bounds: [0.0, 1.0]
    x4:
        type: range
        bounds: [0.0, 1.0]
    x5:
        type: range
        bounds: [0.0, 1.0]
    x6:
        type: range
        bounds: [0.0, 1.0]
    x7:
        type: range
        bounds: [0.0, 1.0]
# These are all defaults, so we don't need to specify them in this case
#script_options:
#    wrapper_path: ./wrapper.py
#    wrapper_name: Wrapper
#    working_dir: .
#    experiment_dir: ... # this is where boa will write logs to by default
                         # if not specified it will be working_dir/experiment_name
#    append_timestamp: True
# This last option 

we need the config normalized, which modifies the parameter section
into a less user friendly form, but what the downstream libraries need

In [13]:
config = boa.load_jsonlike(config_path)

## Define Our Wrapper

We define our wrapper in wrapper.py and use a synthetic function that stands in for any black box model call

In [14]:
with open(Wrapper._path, 'r') as f:
    file_contents = f.read()
    print (file_contents)

import numpy as np
from ax.utils.measurement.synthetic_functions import from_botorch
from botorch.test_functions.synthetic import Cosine8
import boa

cosine8 = from_botorch(Cosine8())


def black_box_model(X) -> float:
    result = -cosine8(X)
    return result

class Wrapper(boa.BaseWrapper):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.data = {}

    def run_model(self, trial) -> None:
        X = np.array([parameter for parameter in trial.arm.parameters.values()])
        # This is a silly toy function, in reality,
        # you could instead import your model main() function and use that, and then collect the results.
        # You could also call an external script to start a model run from Bash or elsewhere.
        self.data[trial.index] = black_box_model(X)

    def set_trial_status(self, trial) -> None:
        data_exists = self.data.get(trial.index)
        if data_exists:
            trial.mark_completed()

    def fetch_tr

## Initialize our Setup

In [15]:
controller = boa.Controller(config_path=config_path, wrapper=Wrapper)

controller.initialize_scheduler()

[INFO 04-25 15:11:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x0. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 04-25 15:11:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x1. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 04-25 15:11:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x2. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 04-25 15:11:57] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter x3. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 04-25 

(Scheduler(experiment=Experiment(boa_runs), generation_strategy=GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 16 trials, GPEI for subsequent trials]), options=SchedulerOptions(max_pending_trials=10, trial_type=<TrialType.TRIAL: 0>, batch_size=None, total_trials=None, tolerated_trial_failure_rate=0.5, min_failed_trials_for_failure_rate_check=5, log_filepath=None, logging_level=20, ttl_seconds_for_trials=None, init_seconds_between_polls=1, min_seconds_before_poll=1.0, seconds_between_polls_backoff_factor=1.5, timeout_hours=None, run_trials_in_batches=False, debug_log_run_metadata=False, early_stopping_strategy=None, global_stopping_strategy=None, suppress_storage_errors_after_retries=False)),
 <wrapper.Wrapper at 0x162aaef80>)

## Run our Experiment

The Controller will save our scheduler to JSON after it completes the run so we can reload it at a later time for analysis or to resume our experiment

In [8]:
scheduler = controller.run()

[INFO 2023-04-25 14:58:01,216 MainProcess] boa: 

##############################################


BOA Experiment Run
Output Experiment Dir: /Users/madelinescyphers/Documents/projs_.nosync/boa/docs/examples/boa_runs_20230425T145759
Start Time: 20230425T145801
Version: 0.8.3.dev1+g0636f51.d20230411

##############################################

[INFO 04-25 14:58:01] Scheduler: Running trials [0]...
[INFO 04-25 14:58:02] Scheduler: Running trials [1]...
[INFO 04-25 14:58:03] Scheduler: Running trials [2]...
[INFO 04-25 14:58:03] Scheduler: Running trials [3]...
[INFO 04-25 14:58:03] Scheduler: Running trials [4]...
[INFO 04-25 14:58:04] Scheduler: Running trials [5]...
[INFO 04-25 14:58:05] Scheduler: Running trials [6]...
[INFO 04-25 14:58:06] Scheduler: Running trials [7]...
[INFO 04-25 14:58:07] Scheduler: Running trials [8]...
[INFO 04-25 14:58:08] Scheduler: Running trials [9]...
[INFO 04-25 14:58:09] Scheduler: Retrieved COMPLETED trials: 0 - 9.
[INFO 04-25 14:58:09] Scheduler: F

In [9]:
## Get the Best Trial and Output All Trials

In [10]:
trial = scheduler.best_raw_trials()
trial

{19: {'params': {'x0': 0.2038856832794238,
   'x1': 0.2732545983784687,
   'x2': 0.4110932205002711,
   'x3': 0.2995378121818915,
   'x4': 0.4110401726876042,
   'x5': 0.2663633438256092,
   'x6': 0.265746840451554,
   'x7': 0.36591015671956817},
  'means': {'Cosine8': 0.7793695510194338},
  'cov_matrix': {'Cosine8': {'Cosine8': 0.0}}}}

In [11]:
boa.scheduler_to_df(scheduler)

,trial_index,arm_name,trial_status,generation_method,Cosine8,x0,x1,x2,x3,x4,x5,x6,x7
0,0,0_0,COMPLETED,Sobol,1.340935,0.077720,0.083882,0.342206,0.690946,0.772082,0.111350,0.471297,0.446977
1,1,1_0,COMPLETED,Sobol,2.375846,0.534666,0.620485,0.367511,0.112688,0.077059,0.691855,0.102911,0.959383
2,2,2_0,COMPLETED,Sobol,2.839811,0.310591,0.975680,0.484553,0.488173,0.728263,0.172154,0.804332,0.182730
3,3,3_0,COMPLETED,Sobol,1.767864,0.312860,0.282408,0.463470,0.070465,0.406250,0.615348,0.765468,0.573685
4,4,4_0,COMPLETED,Sobol,3.416992,0.917239,0.846092,0.087814,0.808420,0.882873,0.330136,0.834354,0.055104
5,5,5_0,COMPLETED,Sobol,1.350048,0.215255,0.519261,0.530978,0.412104,0.504048,0.132552,0.185868,0.369909
6,6,6_0,COMPLETED,Sobol,2.408961,0.380891,0.729300,0.387660,0.014045,0.693759,0.949175,0.137081,0.553832
7,7,7_0,COMPLETED,Sobol,2.047757,0.901627,0.371981,0.336194,0.645727,0.392114,0.812803,0.080181,0.116925
8,8,8_0,COMPLETED,Sobol,2.462431,0.809054,0.357948,0.579874,0.089212,0.766986,0.841262,0.563306,0.018887
9,9,9_0,COMPLETED,Sobol,3.217938,0.491685,0.132569,0.632771,0.733023,0.920983,0.808424,0.568947,0.289268
